In [1]:
import numpy as np
import jdc
import random
import matplotlib.pyplot as plt
import math

In [2]:
class Road_Alignment:
    def __init__(self,data,pop_size):
        self.data = data
        self.shape = data.shape
        self.pop_size = pop_size
        self.population = None

In [3]:
%%add_to Road_Alignment
def initialize_population(self):
    self.population = []
    for i in range(self.pop_size):
        chromosome = self._initialize_chromosome()
        self.population.append(chromosome)

In [4]:
%%add_to Road_Alignment
def _initialize_chromosome(self):
    chromosome = [(-1, -1)] * (self.data.shape[1])
    min_genes = 1
    max_genes = self.data.shape[1]
#     max_genes = 10
    num_genes = random.randint(min_genes, max_genes)
    genes = self._initialize_genes(num_genes)
    chromosome[0] = genes[0] 
    chromosome[-1] = genes[-1] 
    last_index = 0
    for idx, (i, j) in enumerate(genes):
        if idx == 0 or idx == len(genes) -1:
            continue
        next_index = random.randint(last_index + 1, len(chromosome) - len(genes) + idx)
        chromosome[next_index] = (i,j)
        last_index = next_index
    return chromosome

In [5]:
%%add_to Road_Alignment
def _initialize_genes(self, num_genes):
    first_gene = (0, random.randint(0, self.data.shape[0] - 1))
    distinct_tuples = []
    distinct_tuples.append(first_gene)
    count = 0
    while len(distinct_tuples) < num_genes:
        new_tuple = self._initialize_gene(len(distinct_tuples), num_genes)
        if new_tuple[0] > distinct_tuples[-1][0]:
            distinct_tuples.append(new_tuple)
        count += 1
    if distinct_tuples[-1] != (self.data.shape[1] - 1):
        y_coordinate = random.randint(0,self.data.shape[0] - 1)
        distinct_tuples.append((self.data.shape[1] - 1 ,y_coordinate))
    return distinct_tuples

In [6]:
%%add_to Road_Alignment
def _initialize_gene(self, generated_num, num_genes):
    x = random_generation(0, self.data.shape[1] - 1 , generated_num, num_genes)
    y = random_generation(0, self.data.shape[0] - 1 , generated_num, num_genes)
    return x,y

In [7]:
%%add_to Road_Alignment
def random_generation(start, end, numbers, num_numbers):
    
    segment_width = (end - start + 1) / num_numbers
    random_offset = random.uniform(0, segment_width)
    
    random_number = math.ceil(start + (numbers + 1) * segment_width + random_offset)
    return random_number

In [8]:
%%add_to Road_Alignment
def chromosome_genes(self,chromosome):
    genes = []
    for i, (x,y) in enumerate(chromosome):
        if (x,y) != (-1, -1):
            genes.append((x,y))
    return genes

In [9]:
%%add_to Road_Alignment
def calculate_fitness(self,chromosome):
    
    genes = self.chromosome_genes(chromosome)
    total_cost = 0.0 
    for i in range(len(genes) - 1):
        x1, y1 = genes[i]
        x2, y2 = genes[i + 1]
        if abs(x1 - x2) == 1 and abs(y1 - y2) <= 1:
            road_cost = self.data[y1][x1]
            total_cost += road_cost
        else:
            distance_cost = (x2 - x1)**2 + (y2 - y1)**2
            total_cost += distance_cost

    return total_cost

In [10]:
%%add_to Road_Alignment
def parent_tournament_selection(self, tournament_size=5):
    tournament_candidates = random.sample(self.population, tournament_size)
    winner = min(tournament_candidates, key=lambda ind: self.calculate_fitness(ind))
    tournament_candidates = random.sample(self.population, tournament_size)
    runner_up = min(tournament_candidates, key=lambda ind: self.calculate_fitness(ind))
    return winner, runner_up

In [11]:
%%add_to Road_Alignment
def uniform_crossover(self,parent1, parent2, crossover_probability=0.5):
    offspring1 = np.zeros_like(parent1)
    offspring2 = np.zeros_like(parent2)

    for i in range(len(parent1)):
        if np.random.rand() < crossover_probability:
            offspring1[i] = parent1[i]
            offspring2[i] = parent2[i]
        else:
            offspring1[i] = parent2[i]
            offspring2[i] = parent1[i]

    return offspring1, offspring2

In [12]:
%%add_to Road_Alignment
def mutate(self,chromosome,mutation_probability = 0.1):
    mutated_chromosome = np.copy(chromosome)

    for i in range(len(chromosome)):
        if np.random.rand() < mutation_probability:
            x, y = mutated_chromosome[i]
            if i == 0 or i == len(chromosome) - 1:
                y = np.clip(y + np.random.randint(-2, 2), 0, self.data.shape[0] - 1)
                mutated_chromosome[i] = (x, y)
                continue
            if np.random.rand() < 0.5:
                x = np.clip(x + np.random.randint(-5, 5), 0, self.data.shape[1] - 1)
                y = np.clip(y + np.random.randint(-2, 2), 0, self.data.shape[0] - 1)
            else:
                x = -1
                y = -1
            mutated_chromosome[i] = (x, y)
    offspring = list(mutated_chromosome)
    return offspring

In [13]:
%%add_to Road_Alignment
def generate_offspring(self, num_offspring):
    offsprings = []
    while len(offsprings) < num_offspring:
        parent1 = self.parent_tournament_selection()[0]
        parent2 = self.parent_tournament_selection()[1]
        offspring1, offspring2 = self.uniform_crossover(parent1, parent2)
        
        offspring1 = self.mutate(offspring1)
        offspring2 = self.mutate(offspring2)
        offsprings.extend([offspring1, offspring2])
    return offsprings[:num_offspring]

In [14]:
%%add_to Road_Alignment
def elitism_survival_selection(self, fitness_values, offsprings, num_elite):
    sorted_indices = np.argsort(fitness_values)
    elite_indices = sorted_indices[:num_elite]
    elite_individuals = [self.population[i] for i in elite_indices]
    new_generation = elite_individuals + offsprings
    return new_generation

In [15]:
%%add_to Road_Alignment
def terminate(self, generation, max_generations, convergence_threshold, no_improvement_threshold, best_fitness_values):
    if generation >= max_generations:
        return True
    
    if has_converged(best_fitness_values, convergence_threshold):
        return True
    
    if no_improvement(best_fitness_values, no_improvement_threshold):
        return True
    
    return False

In [16]:
def has_converged(best_fitness_values, convergence_threshold):
    if len(best_fitness_values) > 1:
        last_best_fitness = best_fitness_values[-2]
        current_best_fitness = best_fitness_values[-1]
        
        change_in_fitness = np.abs(current_best_fitness - last_best_fitness)
        
        return change_in_fitness < convergence_threshold
    
    return False

In [17]:
def no_improvement(best_fitness_values, no_improvement_threshold):
    if len(best_fitness_values) > no_improvement_threshold:
        recent_best_fitness = best_fitness_values[-no_improvement_threshold:]
        improvement_check = all(recent_best_fitness[i] >= recent_best_fitness[i + 1] for i in range(no_improvement_threshold - 1))   
        return improvement_check
    
    return False
    

In [18]:
%%add_to Road_Alignment
def extract_solution(self,population, fitness_values):
    best_index = np.argmin(fitness_values)
    best_individual = population[best_index]
    return best_individual

In [19]:
%%add_to Road_Alignment
def visualize_map(self):
    plt.imshow(self.data, cmap='YlGn', interpolation='nearest')
    cbar = plt.colorbar()
    plt.gca().invert_yaxis()
    cbar.set_label('Roughness Level')
    plt.title('Map Roughness')
#     plt.show()
    return plt

In [20]:
%%add_to Road_Alignment
def visualize_chromosome(self, plt,chromosome):
    genes = self.chromosome_genes(chromosome)
    for i in range(len(genes) - 1):
        x1, y1 = genes[i]
        x2, y2 = genes[i + 1]
        if abs(x1 - x2) == 1 and abs(y1 - y2) <= 1:
            plt.plot([x1, x2], [y1, y2], marker='o', linestyle='-',color='black')
        else:
            plt.plot([x1, x2], [y1, y2], marker='o', linestyle='-',color='red')

    plt.title('chromosome')
    plt.show()

In [21]:
data = np.load('map.npy')
ra = Road_Alignment(data,7)
ra.initialize_population()

ValueError: empty range for randrange() (1, 1, 0)

In [ ]:
max_generations = 300
tournament_size = 5
crossover_probability = 0.7
mutation_probability = 0.1
convergence_threshold = 1e-6
no_improvement_threshold = 10
elite_percentage = 0.1

best_fitness_values = []
count = 0
for generation in range(max_generations):
    
    fitness_values = [ra.calculate_fitness(ind) for ind in ra.population]
    best_fitness_values.append(min(fitness_values))
    num_elite = int(len(ra.population) * elite_percentage)
    offsprings = ra.generate_offspring(len(ra.population) - num_elite)
    ra.elitism_survival_selection(fitness_values, offsprings, num_elite)
    
    # Check for termination conditions
#     if ra.terminate(generation, max_generations, convergence_threshold, no_improvement_threshold, best_fitness_values):
#         break
    # Visualize the best individual in the current generation
    
best_individual = ra.extract_solution(ra.population, fitness_values)
print(ra.calculate_fitness(best_individual))
plt = ra.visualize_map()
ra.visualize_chromosome(plt, best_individual)


In [25]:
[(i,random.randint(0,19)) for i in range(0,60)]

[(0, 1),
 (1, 17),
 (2, 9),
 (3, 16),
 (4, 15),
 (5, 2),
 (6, 12),
 (7, 6),
 (8, 9),
 (9, 14),
 (10, 11),
 (11, 1),
 (12, 4),
 (13, 2),
 (14, 17),
 (15, 16),
 (16, 0),
 (17, 18),
 (18, 18),
 (19, 12),
 (20, 8),
 (21, 6),
 (22, 4),
 (23, 12),
 (24, 14),
 (25, 9),
 (26, 10),
 (27, 3),
 (28, 10),
 (29, 13),
 (30, 10),
 (31, 17),
 (32, 4),
 (33, 0),
 (34, 7),
 (35, 5),
 (36, 1),
 (37, 4),
 (38, 12),
 (39, 12),
 (40, 14),
 (41, 2),
 (42, 7),
 (43, 11),
 (44, 5),
 (45, 8),
 (46, 12),
 (47, 1),
 (48, 15),
 (49, 10),
 (50, 6),
 (51, 6),
 (52, 12),
 (53, 13),
 (54, 2),
 (55, 2),
 (56, 5),
 (57, 9),
 (58, 18),
 (59, 5)]